In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn import metrics
import joblib                                                                                                                                            
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import auc
from sklearn.preprocessing import MinMaxScaler

from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from scipy.stats import pearsonr



In [12]:
fajiao = pd.read_table("../data-2/fajiao-condition-4.11.txt",sep='\t',index_col=0)
fajiao = pd.get_dummies(fajiao, columns=fajiao.columns) 
fajiao['label'] = fajiao.groupby(fajiao.columns.tolist()).ngroup()

fajiao_drop = fajiao.drop_duplicates()
fajiao_drop = fajiao_drop.drop('label', axis=1)

X = np.array(fajiao_drop)

# X  = StandardScaler().fit_transform(X)
X  = Normalizer().fit_transform(X)
# # X  = MinMaxScaler().fit_transform(X)
# X  = StandardScaler().fit_transform(X)
X.shape

(13, 17)

In [13]:
data = pd.read_table('../data-2/total4.15.txt',sep='\t',index_col=0)
data_fjg= data.loc[:, ~data.columns.str.contains('BC|NT|MC|PC')].T
data_fjg.columns = data_fjg.iloc[0]
data_fjg = data_fjg[1:]
data_fjg = data_fjg.sort_index()
data_fjg = data_fjg.replace("#DIV/0!", 0)
data_fjg = data_fjg.apply(pd.to_numeric)
data_fjg.fillna(0, inplace=True)
# data_fjg

data_fjg.index = data_fjg.index.astype(float) 
data_fjg = data_fjg.sort_index() 
selected_columns = ['4602-84-0', '590-86-3', '1014-60-4', '6114-18-7', '123-69-3', '1000152-34-3', '1000377-93-5']
data_fjg = data_fjg.loc[:, selected_columns]
# data_fjg

data_fjg['label'] = fajiao['label']
fajiao_mean = data_fjg.groupby('label').mean()

fajiao_mean_unique = fajiao_mean.reset_index().drop_duplicates()
fajiao_mean_unique = fajiao_mean_unique.drop('label', axis=1)
fajiao_mean_unique

Y = np.array(fajiao_mean_unique)
# 
Y  = Normalizer().fit_transform(Y)
# Y  = MinMaxScaler().fit_transform(Y)
# Y  = StandardScaler().fit_transform(Y)
# Y

In [14]:
fajiao_mean_unique


ID,4602-84-0,590-86-3,1014-60-4,6114-18-7,123-69-3,1000152-34-3,1000377-93-5
0,1.650409e+06,4.925178e+05,2.383041e+05,1.912917e+06,0.00,184010.225000,4.751404e+05
1,7.581497e+06,2.019396e+06,1.392228e+06,5.482815e+06,1518022.25,0.000000,5.293125e+05
2,8.761448e+06,1.240506e+06,1.418163e+06,8.114793e+06,0.00,116849.541100,5.049237e+05
3,4.817076e+06,1.050808e+06,1.595233e+06,5.133663e+06,0.00,115235.365000,1.057956e+06
4,8.434836e+05,5.245687e+05,6.910791e+05,1.658898e+06,0.00,204461.600666,1.154414e+05
5,1.283222e+07,1.554018e+06,0.000000e+00,1.137263e+07,598614.25,172899.500000,9.361930e+05
6,4.001551e+06,8.425780e+05,1.239685e+06,4.900436e+06,912470.00,62543.380666,5.157412e+05
7,1.037140e+07,1.708906e+06,0.000000e+00,1.066899e+07,727370.25,140610.875000,8.042812e+05
8,2.213889e+06,6.043978e+05,1.220957e+06,2.927182e+06,0.00,97112.960000,1.192698e+05
9,4.793210e+06,1.167713e+06,3.831642e+05,8.172780e+06,0.00,79998.544000,5.918712e+05


In [15]:
base_model = RandomForestRegressor(n_estimators=3, random_state=42)
gbr_model = MultiOutputRegressor(base_model)

In [16]:
gbr_model = MultiOutputRegressor(base_model)

cv = KFold(n_splits=5, shuffle=True, random_state=42)

MSE = []
RMSE = []
MAE = []
PCC = []
R2 = []
adjusted_r2 = []

MSE_train = []
RMSE_train = []
MAE_train = []
PCC_train = []
R2_train = []
adjusted_r2_train = []


for train_idx, test_idx in cv.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, y_test = Y[train_idx], Y[test_idx]
    gbr_model.fit(X_train, Y_train)
    # print(X_train.shape, Y_train.shape)
    #train
    x_pred = gbr_model.predict(X_train)
    mse = mean_squared_error(Y_train, x_pred)
    rmse = np.sqrt(mean_squared_error(Y_train, x_pred))
    mae = np.mean(np.abs(Y_train - x_pred))
    correlation_array = np.empty((len(Y_train), 1))
    for i in range(len(Y_train)):
        correlation = np.corrcoef(Y_train[i], x_pred[i])[0, 1]
        correlation_array[i] = correlation
    pcc = np.mean(correlation_array)
    r2 = r2_score(Y_train, x_pred)
    num_features = 6
    adj_r2 = 1 - (1 - r2) * (len(Y_train) - 1) / (len(Y_train) - num_features - 1)
    MSE_train .append(mse)
    RMSE_train .append(rmse)
    MAE_train .append(mae)
    PCC_train .append(pcc)
    R2_train .append(r2)
    adjusted_r2_train .append(adj_r2)

    #test
    y_pred = gbr_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = np.mean(np.abs(y_test - y_pred))
    correlation_array = np.empty((len(y_test), 1))
    for i in range(len(y_test)):
        correlation = np.corrcoef(y_test[i], y_pred[i])[0, 1]
        correlation_array[i] = correlation
    pcc = np.mean(correlation_array)
    r2 = r2_score(y_test, y_pred) 
    num_features = 6
    adj_r2 = 1 - (1 - r2) * (len(y_test) - 1) / (len(y_test) - num_features - 1)
    MSE.append(mse)
    RMSE.append(rmse)
    MAE.append(mae)
    PCC.append(pcc)
    R2.append(r2)
    adjusted_r2.append(adj_r2)


for i, (a1,a2,a3,a4,a5,a6) in enumerate(zip(MSE_train, RMSE_train,MAE_train, PCC_train,R2_train,adjusted_r2_train), 1):
    print(f'Fold {i}: MSE = {a1:.4f}, RMSE = {a2:.4f},MAE = {a3:.4f}, PCC = {a4:.4f}, R2 = {a5:.4f}, adjusted_r2 = {a6:.4f}')


average_MSE= sum(MSE_train) / len(MSE_train)
average_RMSE = sum(RMSE_train) / len(RMSE_train)
average_MAE= sum(MAE_train) / len(MAE_train)
average_PCC = sum(PCC_train) / len(PCC_train)
average_R2 = sum(R2_train) / len(R2_train)
average_adjusted_r2 = sum(adjusted_r2_train) / len(adjusted_r2_train)

print(f'Average MSE_train = {average_MSE:.7f}')
print(f'Average RMSE_train = {average_RMSE:.7f}')
print(f'Average MAE_train = {average_MAE:.7f}')
print(f'Average PCC_train = {average_PCC:.7f}')
# print(f'Average R2_train = {average_R2:.7f}')
# print(f'Average adjusted_R2_train = {average_adjusted_r2:.7f}')



for i, (a1,a2,a3,a4,a5,a6) in enumerate(zip(MSE, RMSE,MAE, PCC,R2,adjusted_r2), 1):
    print(f'Fold {i}: MSE = {a1:.4f}, RMSE = {a2:.4f},MAE = {a3:.4f}, PCC = {a4:.4f}, R2 = {a5:.4f}, adjusted_r2 = {a6:.4f}')

average_MSE= sum(MSE) / len(MSE)
average_RMSE = sum(RMSE) / len(RMSE)
average_MAE= sum(MAE) / len(MAE)
average_PCC = sum(PCC) / len(PCC)
average_R2 = sum(R2) / len(R2)
average_adjusted_r2 = sum(adjusted_r2) / len(adjusted_r2)

print(f'Average MSE = {average_MSE:.7f}')
print(f'Average RMSE = {average_RMSE:.7f}')
print(f'Average MAE = {average_MAE:.7f}')
print(f'Average PCC = {average_PCC:.7f}')
# print(f'Average R2 = {average_R2:.7f}')
# print(f'Average adjusted_R2 = {average_adjusted_r2:.7f}')

Fold 1: MSE = 0.0071, RMSE = 0.0844,MAE = 0.0494, PCC = 0.9606, R2 = 0.3844, adjusted_r2 = -0.8469
Fold 2: MSE = 0.0070, RMSE = 0.0834,MAE = 0.0478, PCC = 0.9534, R2 = 0.2876, adjusted_r2 = -1.1373
Fold 3: MSE = 0.0055, RMSE = 0.0740,MAE = 0.0418, PCC = 0.9675, R2 = 0.3799, adjusted_r2 = -0.8604
Fold 4: MSE = 0.0026, RMSE = 0.0512,MAE = 0.0280, PCC = 0.9828, R2 = 0.4763, adjusted_r2 = -0.3094
Fold 5: MSE = 0.0049, RMSE = 0.0701,MAE = 0.0379, PCC = 0.9706, R2 = 0.5533, adjusted_r2 = -0.1168
Average MSE_train = 0.0054177
Average RMSE_train = 0.0726186
Average MAE_train = 0.0409723
Average PCC_train = 0.9669744
Fold 1: MSE = 0.0091, RMSE = 0.0952,MAE = 0.0661, PCC = 0.9465, R2 = -1.7793, adjusted_r2 = 2.3897
Fold 2: MSE = 0.0380, RMSE = 0.1949,MAE = 0.1327, PCC = 0.7740, R2 = -31.9501, adjusted_r2 = 17.4751
Fold 3: MSE = 0.0130, RMSE = 0.1138,MAE = 0.0881, PCC = 0.9146, R2 = -11.8446, adjusted_r2 = 7.4223
Fold 4: MSE = 0.0214, RMSE = 0.1463,MAE = 0.0944, PCC = 0.8722, R2 = -118.2084, adju

In [21]:

yp = gbr_model.predict(X)
# np.savetxt('my_array.txt', yp, delimiter='\t')
yp.shape  


yp  = Normalizer().fit_transform(yp)
yp = MinMaxScaler().fit_transform(yp)
print(yp.shape)
loaded_model = joblib.load("../code-2/result/RFC_Classifier-7f.joblib")
y_pred = loaded_model.predict(yp).tolist() 
y_pred

(13, 7)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [18]:
df13 = fajiao.drop_duplicates()

df54 = pd.read_csv("fjg_label.csv",index_col=0)
df54['code'] = fajiao['label']
df54['mean'] = df54.groupby('code')['label'].transform('mean')
df54['mean_label'] = np.where(df54['mean'] > 0.6, 1, 0)
df54

df54 = df54[df54.index.isin(df13.index)]
truth_simi = df54['mean_label'].tolist()
truth_simi

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

In [19]:
print('precision（test）:',metrics.precision_score(truth_simi,y_pred))
print('recall（test）:',metrics.recall_score(truth_simi,y_pred))
print('Accuracy（test）:',metrics.accuracy_score(truth_simi,y_pred))
# print('F1 （test）:',f1_score(truth_simi,y_pred))
print("y_test:",truth_simi,"\ny_pred:",y_pred)
fpr, tpr, thresholds = roc_curve(truth_simi, y_pred)
auc_1 = auc(fpr, tpr)
print("AUC : ", auc_1)

precision（测试集）: 0.0
recall（测试集）: 0.0
Accuracy（测试集）: 0.8461538461538461
y_test: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0] 
y_pred: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
AUC :  0.4583333333333333


In [20]:
yp
result = pd.DataFrame(yp,index = fajiao_drop.index,columns=selected_columns)
result

,4602-84-0,590-86-3,1014-60-4,6114-18-7,123-69-3,1000152-34-3,1000377-93-5
4.2,0.795739,0.446107,0.694889,0.013757,0.000000,0.841944,0.945054
5.2,0.932923,0.142967,0.171273,0.081973,0.207586,0.017564,0.586215
6.2,1.000000,0.121707,0.284029,0.000000,0.000000,0.011950,0.477018
16.1,0.480901,0.549369,0.732023,0.288892,0.000000,1.000000,1.000000
17.1,0.254330,0.540217,1.000000,0.469427,0.000000,0.987152,0.569426
23.1,0.033435,0.999996,0.480185,0.531344,0.000000,0.021696,0.122620
24.1,0.033440,1.000000,0.480187,0.531356,0.000000,0.018069,0.122622
25.1,0.586391,0.325408,0.100649,0.444766,0.211117,0.023270,0.163125
26.1,0.418188,0.373248,0.692562,0.514861,0.000000,0.088867,0.049701
27.1,0.225425,0.168887,0.000000,0.810127,0.000000,0.000000,0.207686
